In [ ]:
import tweepy
import webbrowser
import time
import pprint
import json
import pandas as pd
from io import StringIO
import datetime
import nltk
import pycld2 as cld2
from nltk.tokenize import TweetTokenizer
import re
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans, MiniBatchKMeans
from  sklearn.metrics import fowlkes_mallows_score



nltk.download(["names","stopwords","state_union","twitter_samples","movie_reviews","averaged_perceptron_tagger","vader_lexicon","punkt"])

In [ ]:
# API KEY
consumer_key = 'y8oMppAVPw9DpK3juBkTZoDvs'

# API SECRET
consumer_secret =  'tIWtYPtu6UhSpCeZcjeerWEk7jrPI0O9ks9YijRu5x4lN6kWPl'

# BEARER TOKEN
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJykaQEAAAAAUng82dZX%2BsucZhLqh6EqRDQPnVw%3DCghiwIoSP4dX3xZzcJJofXtwKwVvhWO45fffJOmd13z4aqPLow'

callback_uri = 'oob'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback=callback_uri)

In [ ]:
redirect_url = auth.get_authorization_url()
print(redirect_url)

In [ ]:
webbrowser.open(redirect_url)

In [ ]:
user_pin = '9932719'
auth.get_access_token(user_pin)

In [ ]:
api = tweepy.API(auth)

In [ ]:
api.get_user(screen_name='code')

In [ ]:
tweets_json = api.search_30_day('dev', query='Ukraine lang:en (place_country:US OR place_country:CA)')

In [ ]:
tweets_json_db =  [json.dumps(t._json) for t in tweets_json]

In [ ]:
# Reads the tweet's JSON info and turns it into a dataframe for cleaning
tweets_df = pd.concat(pd.json_normalize(tweets_json[i]._json) for i, t in enumerate(tweets_json))

In [ ]:
def tweet_to_df(tweet_json):
    """
    returns dataframe of tweets from list of JSON files
    """
    return pd.concat(pd.json_normalize(tweet_json[i]._json) for i, t in enumerate(tweet_json))

In [ ]:
tweets_df.head()

In [ ]:
# Output data as .csv file for easier sharing
tweets_df.to_csv('tweets_df_raw.csv')

In [ ]:
# Looking at all 187 column names
for c in tweets_df.columns:
    print(c)

In [ ]:
tweets_df['created_at'].min()

In [ ]:
# How to build a query
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/guides/build-standard-query
tweets2 = api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', fromDate='202202210000', toDate='202202280000', maxResults=100)

In [ ]:
df2 = tweet_to_df(tweets2)

In [ ]:
# Sorry for the ugly code, but it works
day = [21, 22, 23, 24, 25, 26, 27, 28, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
day = [str(i) for i in day]
hour = ['00', '06', '12', '18']

In [ ]:
data = []
for d in day:
    for h in hour:
        data.append(api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', toDate=f'202202{d}{h}00', maxResults=10))
data

In [ ]:
new_data = pd.DataFrame()
test = [tweet_to_df(data[i]) for i in range(len(data))]
for t in test:
    new_data = pd.concat([new_data, t])
    
new_data

In [ ]:
new_data.to_csv('new_data.csv')

In [ ]:
feb28_midnight = api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', toDate=f'202202280000', maxResults=100)

In [ ]:
day = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

In [ ]:
march_data = []

for d in day:
    for h in hour:
        march_data.append(api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', toDate=f'202203{d}{h}00', maxResults=100))
march_data

In [ ]:
test2 = [tweet_to_df(march_data[i]) for i in range(len(march_data))]

In [ ]:
march_tweets = pd.DataFrame()
for i in range(len(test2)):
    march_tweets = pd.concat([march_tweets, test2[i]])

In [ ]:
march_tweets.to_csv('march01_march08.csv')

In [ ]:
def combine_tweet_dfs(tweet_data):
    """
    Returns dataframe of concatenated tweet dataframes
    """
    tweet_dfs = [tweet_to_df(tweet_data[i]) for i in range(len(tweet_data))]
    output_df = pd.DataFrame()
    for i in range(len(tweet_dfs)):
        output_df = pd.concat([output_df, tweet_dfs[i]])
        
    return output_df

In [ ]:
march_data_09_on = []

day = ['09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

for d in day:
    for h in hour:
        march_data_09_on.append(api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', toDate=f'202203{d}{h}00', maxResults=100))
march_data_09_on

In [ ]:
# Just to help me know when I can call on the API again
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
march_data_09_onwards_df = combine_tweet_dfs(march_data_09_on)

In [ ]:
march_data_09_onwards_df.to_csv('march09_march16.csv')

In [ ]:
set(march_data_09_onwards_df.columns) - set(tweets_df.columns)

In [ ]:
march_data_16_on = []

day = ['16', '17', '18', '19', '20']

for d in day:
    for h in hour:
        march_data_16_on.append(api.search_30_day('dev', query='Ukraine OR Putin OR Zelensky OR Russia lang:en (place_country:CA)', toDate=f'202203{d}{h}00', maxResults=100))
march_data_16_on

In [ ]:
march_data_16_onwards_df = combine_tweet_dfs(march_data_16_on)

In [ ]:
march_data_16_onwards_df.to_csv('march16_march20.csv')

In [ ]:
df1 = pd.read_csv('march01_march08.csv')
df2 = pd.read_csv('march09_march16.csv')
df3 = pd.read_csv('march16_march20.csv')
full_df = pd.concat([df1, df2, df3])
full_df

In [ ]:
for c in  full_df.columns:
    print(c)

In [ ]:
keep = []

for c in full_df.columns:
    if c.startswith(('user', 'retweeted_status', 'quoted_status')):
        continue
    else:
        keep.append(c)
keep
keep = keep[1:]
full_df_new = full_df[keep]
full_df_new.isna().all()
full_df_new.drop(columns=['geo', 'coordinates', 'place', 'contributors'], axis=1, inplace=True)
full_df_new.reset_index(drop=True, inplace=True)
full_df_new.to_csv('Ukraine_tweets.csv')

In [ ]:
df1 = pd.read_csv("Ukraine_tweets.csv")
df = df1
def decode_uni(string):
    string = string.split()
    stringdecoded = ""
    for word in string:
        if "https" in word:
            tempstring = word
        else:
            word = word.encode("ascii", "ignore")
            word = str(word.decode())
            tempstring = ""
            for character in word:
                if str(character).isalnum() or ("." in str(character)) or ("," in str(character)) or ("(" in str(character)) or (")" in str(character)) or (":" in str(character)):
                    tempstring = tempstring+(str(character))
        stringdecoded = stringdecoded+ tempstring + " "
        temp = stringdecoded.split()
        stringdecoded = ""
        for i in range(len(temp)):
            stringdecoded = stringdecoded+temp[i]+" "
    return stringdecoded
#df = df.applymap(decode_uni)
#print(df.head())
df['text'] = pd.DataFrame(df['text']).applymap(decode_uni)
df = df.dropna(axis = 1)
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop('quote_count', axis = 1)
print(df)

In [ ]:
def find_polarity(topics,tweets):
    """Returns two lists, one with negative polarity tweets and one 
    with positive polarity tweets"""

    dfpos = pd.DataFrame()
    dfneg = pd.DataFrame()
    dfneu = pd.DataFrame()
    indexpos = -1
    indexneg = -1
    indexneu = -1

    for tweet in tweets:
        # Initialize first look at tweet
        first = True
        ind = 1
        for topic in topics:
            # Checks if topic mentioned in tweet
            if topic in tweet or topic.upper() in tweet or topic.lower() in tweet \
            or "#"+topic in tweet or "#"+topic.upper() in tweet or "#"+topic.lower() in tweet:
                # First time looking at tweet, add first topic id and polarity scores
                # Positive polarity
                if (sia.polarity_scores(tweet)['compound'] > 0):
                    if (first == True):
                        pos = {'tweet': tweet, 'topic_'+str(ind): topic, 
                        'compound_score': sia.polarity_scores(tweet)['compound'], 
                        'positive_score':sia.polarity_scores(tweet)['pos'], 
                        'negative_score': sia.polarity_scores(tweet)['neg'],'topic_2':'',
                        'topic_3':'','topic_4':'','topic_5':'','topic_6':'','topic_7':''}
                        dfpos = dfpos.append(pos, ignore_index = True)
                        first = False
                        ind = ind+1
                        indexpos+=1
                    else:
                        dfpos.at[indexpos, 'topic_'+str(ind)] = topic
                        ind = ind+1
                    # If not first pass, only add new topics to 'topic' column
                    # Negative polarity
                elif(sia.polarity_scores(tweet)==0):
                    if (first == True):
                        neu = {'tweet': tweet, 'topic_'+str(ind): topic, 
                        'compound_score': sia.polarity_scores(tweet)['compound'], 
                        'positive_score':sia.polarity_scores(tweet)['pos'], 
                        'negative_score': sia.polarity_scores(tweet)['neg'],'topic_2':'',
                        'topic_3':'','topic_4':'','topic_5':'','topic_6':'','topic_7':''}
                        dfneu = dfneu.append(neg, ignore_index = True)
                        first = False
                        ind = ind+1
                        indexneu+=1
                    else:
                        dfneu.at[indexneg, 'topic_'+str(ind)] = topic
                        ind = ind+1
                else:
                    if (first == True):
                        neg = {'tweet': tweet, 'topic_'+str(ind): topic, 
                        'compound_score': sia.polarity_scores(tweet)['compound'], 
                        'positive_score':sia.polarity_scores(tweet)['pos'], 
                        'negative_score': sia.polarity_scores(tweet)['neg'],'topic_2':'',
                        'topic_3':'','topic_4':'','topic_5':'','topic_6':'','topic_7':''}
                        dfneg = dfneg.append(neg, ignore_index = True)
                        first = False
                        ind = ind+1
                        indexneg+=1
                    else:
                        dfneg.at[indexneg, 'topic_'+str(ind)] = topic
                        ind = ind+1
                    
    # If not first pass, only add new topics to 'topic' column
    # Indicates first pass is over
    return dfpos, dfneg, dfneu

In [ ]:
topics = ['Putin','Ukraine','Ukrainian','NATO','Kyiv','Kiev','Zelensky','SWIFT','Russian',
          'Russia','UkraineRussianWar','StandWithUkraine️']
topics = topics.sort()

df_pos = pd.DataFrame()
df_neg = pd.DataFrame()
df_neu = pd.DataFrame()

pos, neg, neu = find_polarity(topics,tweets)
df_pos = pd.concat([df_pos, pos])
df_neg = pd.concat([df_neg, neg])
df_neu = pd.concat([df_neu, neu])
df_pos['sentiment'] = 1
df_neg['sentiment'] = -1
df_neu['sentiment'] = 0
fulldf = pd.concat([df_pos, df_neg, df_neu])
fulldf.to_csv("fulldf.csv")

In [ ]:
#Checks for and returns only english tweets

df = pd.read_csv('fulldf.csv')
alltweets = df['tweet']

tweets = pd.DataFrame()
lang = []

for tweet in alltweets:
    lang.append(cld2.detect(tweet)[2][0][1])

df['lang'] = lang

df = df.loc[df['lang']=='en']

df.to_csv('TweetsByTopic_EN.csv')

In [ ]:
data = pd.read_csv('TweetsByTopic_EN.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data['topic_1'].replace('Ukrainian', 'Ukraine', inplace=True)
data['topic_1'].replace('Russian', 'Russia', inplace=True)
data

In [ ]:
data['tweet'] = data['tweet'].apply(str.lower)

stopwords = nltk.corpus.stopwords.words("english")
tt = TweetTokenizer()
tokens = data['tweet'].apply(tt.tokenize)

def remove_stopwords(lst):
    return ' '.join(word for word in lst if word not in stopwords)

new_X = tokens.apply(remove_stopwords)

In [ ]:

replaceDict = dict({
'{':" ", '}':" ", ',':"", '.':" ", '!':" ", '\\':" ", '/':" ", '$':" ", '%':" ",
'^':" ", '?':" ", '\'':" ", '"':" ", '(':" ", ')':" ", '*':" ", '+':" ", '-':" ",
'=':" ", ':':" ", ';':" ", ']':" ", '[':" ", '`':" ", '~':" ",
})

rep = dict((re.escape(k),v) for k, v in replaceDict.items())
pattern = re.compile('|'.join(rep.keys()))
def replacer(text):
    return rep[re.escape(text.group(0))]

words = data.tweet.str.replace(pattern, replacer).str.lower().str.split()
# words = pd.DataFrame(words.tolist())
words

In [ ]:
ps = PorterStemmer()

# stemmed_words = [ps.stem(w) for lst in words for w in lst]
def get_stems(lst):
    return ' '.join(ps.stem(w) for w in lst)

In [ ]:
stemmed_words = words.apply(get_stems)
data['stemmed_text'] = stemmed_words

In [ ]:
vectorizer = CountVectorizer()
tf = TfidfTransformer()

X = stemmed_words
y = data['topic_1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)


X_train = vectorizer.fit_transform(X_train) #fit_transform CountVectorizer on training data
X_train = tf.fit_transform(X_train)         #fit_transform TfidfTransformer on training data

X_test = vectorizer.transform(X_test)       #transform CountVectorizer on testing data
X_test = tf.transform(X_test)               #transform TfidfTransformer on testing data

In [ ]:
for k in range(1, 11):
     km = MiniBatchKMeans(
         n_clusters=k,
         init="k-means++",
         n_init=1,
         init_size=1000,
         batch_size=1000,
     )
    
    km.fit(X_train)
    preds = km.predict(X_test)
    print(pd.crosstab(y_test, preds))
    print(k, sklearn.metrics.fowlkes_mallows_score(y_test, preds))

In [ ]:
km = MiniBatchKMeans(
        n_clusters=3,
        init="k-means++",
        n_init=1,
        init_size=1000,
        batch_size=1000,
        random_state=3
    )


In [ ]:
km.fit(X_train)
preds = km.predict(X_test)
pd.crosstab(y_test, preds)

In [ ]:
map_war = {2:"Russia", 0:"Putin", 1:"Ukraine"}

In [ ]:
predicted_test = pd.Series(preds).map(map_war)
# predicted_test
sklearn.metrics.accuracy_score(y_test, predicted_test)

In [ ]:
sklearn.metrics.fowlkes_mallows_score(y_test, predicted_test)

In [ ]:
from sklearn.decomposition import TruncatedSVD
 
tsvd = TruncatedSVD(2) # we need 2 principal components to use a scatterplot
converted_data_train = tsvd.fit_transform(X_train)
converted_data_test = tsvd.transform(X_test)

In [ ]:
colors = [i for i in km.labels_]

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize = (10,6))
c_map = plt.cm.get_cmap('jet', 3)
plt.scatter(converted_data_test[:, 0], converted_data_test[:, 1], s = 15,
            cmap = c_map, c = preds)
plt.colorbar()
plt.xlabel('PC-1') , plt.ylabel('PC-2')
plt.title('Clustering Tweets (K=3)')
plt.show()

In [ ]:
#Fitting the whole dataframe, but unused in project
X = stemmed_words
y = data['topic_1']

X2 = vectorizer.fit_transform(X)
X2 = tf.fit_transform(X2)

km.fit(X2)
preds = km.predict(X2)
# pd.crosstab(y, preds)
sklearn.metrics.fowlkes_mallows_score(y, preds)
#looking at tweets by cluster
data['preds'] = preds
data

In [ ]:
sample_cluster0 = data[data['preds'] == 0].sample(100)
for t in sample_cluster0['tweet']:
    print(t)

In [ ]:
sample_cluster1 = data[data['preds'] == 1].sample(100)
for t in sample_cluster1['tweet']:
    print(t)

In [ ]:
sample_cluster2 = data[data['preds'] == 2].sample(100)
for t in sample_cluster2['tweet']:
    print(t)

In [ ]:
 
tsvd = TruncatedSVD(3) # we need 2 principal components to use a scatterplot
converted_data_train = tsvd.fit_transform(X_train)
converted_data_test = tsvd.transform(X_test)

In [ ]:
#3D Scatterplot
data['keyword'] = data.topic_1.map({'Ukraine': 1, 'Rusia': 2, 'Zelensky': 3, 'Putin':4}).fillna(0).astype(int)
x=data.positive_score
y=data.negative_score
z=data.compound_score
print(x.shape)
print(y.shape)
print(z.shape)

fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot(projection='3d')
plt.style.use('seaborn-whitegrid')
c_map = plt.cm.get_cmap('jet', 10)
ax.scatter(x, y, z, s = 25,
            cmap = c_map, c = data['keyword'], label = {0:"Ukraine", 1:"Russia", 2:"Russian", 3:"Putin"})
ax.set_xlabel('Positive') , ax.set_ylabel('Negative'), ax.set_zlabel('Compound')
plt.title('Scatterplot')
plt.legend()
plt.show()
plt.savefig('3DScatter.png',dpi=100)



In [ ]:
#3D Clustering
x=converted_data_test[:, 0]
y=converted_data_test[:, 1]
z=converted_data_test[:,2]
print(x.shape)
print(y.shape)
print(z.shape)

fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot(projection='3d')
plt.style.use('seaborn-whitegrid')
c_map = plt.cm.get_cmap('jet', 10)
ax.scatter(x, y, z, s = 25,
            cmap = c_map, c = data['keyword'], label = {0:"Ukraine", 1:"Russia", 2:"Russian", 3:"Putin"})
ax.set_xlabel('PC-1') , ax.set_ylabel('PC-2'), ax.set_zlabel('PC-3')
plt.title('Clustering Tweets (K=4)')
plt.legend()
plt.show()
plt.savefig('3DCluster.png',dpi=100)